# PydanticAI example with LiteLLM Proxy and SAP LLMs

## [How PydanticAI works](https://ai.pydantic.dev/agents/)
## [LiteLLM Proxy setup for SAP models](../proxy_set_up.ipynb)

## Installation

In [ ]:
%pip install pydantic-ai litellm

## Set env variables
Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
"PROXY_MASTER_KEY": sk-1234,
"PROXY_BASE_URL": "http://localhost:4000"
```

## Run the PydanticAI with LiteLLM and SAP LLMs

In [ ]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.litellm import LiteLLMProvider
from dotenv import load_dotenv
import os
import litellm

Load your credentials as environment variables.

In [ ]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("LITELLM_ROXY_URL")
api_key = os.getenv("LITELLM_MASTER_KEY")

Define the model with OpenAIChatModel object with LiteLLMProvider object with your proxy credentials

In [ ]:
model = OpenAIChatModel(
    "sap/gpt-5",
    provider=LiteLLMProvider(
        api_base=api_base,
        api_key=api_key,
    ),
)

Define the agent

In [ ]:
agent = Agent(
    model=model,
    system_prompt="You are a helpful weather assistant. "
    "When the user send you asks a specific city, "
    "use the 'get_weather' tool to find the information about the weather. "
    "Aser with TV weather report in two sentences, include small jok",
)

Define the agent tool.

In [ ]:
@agent.tool
def get_weather(city: RunContext[str]) -> str:
    """Mock function"""
    city_normalized = city.prompt.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Run agent

In [ ]:
result = agent.run_sync("London")
print(result.output)